# Prepare mitosis time series data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic

In [2]:
sample_json_dir = Path("./EBSS_starvation_24h_xy16_annotation")
sample_dataset_dir = sample_json_dir / "datasets"
class_subfolders = ["mitosis", "apoptosis", "normal"]
# sample_paths = glob.glob(str(sample_json_dir / "*.json"))

class_samples = {}
for subfolder in class_subfolders:
    class_samples[subfolder] = []
    sample_paths = glob.glob(str(sample_json_dir / subfolder / "*.json"))
    for sample_path in sample_paths:
        sample = SingleCellStatic.load_single_cells_json(sample_path)
        class_samples[subfolder].append(sample)

In [3]:
class_samples

{'mitosis': [[SingleCellStatic(id=af619ea2-3c12-4fc6-aec2-7192ee4ecdab, timeframe=83, bbox=[1307. 1065. 1369. 1123.]),
   SingleCellStatic(id=74a36316-dc3c-46df-8c4c-d90600ff52d8, timeframe=83, bbox=[1377. 1057. 1434. 1117.]),
   SingleCellStatic(id=a9459527-8e30-4413-9178-cdf7f95505f9, timeframe=82, bbox=[1304. 1073. 1373. 1134.]),
   SingleCellStatic(id=603ca0a7-ba26-47fe-bd1d-d83ec57a45a8, timeframe=82, bbox=[1373. 1058. 1427. 1115.]),
   SingleCellStatic(id=0a62dd51-491c-4acb-9a43-2e250fec4794, timeframe=81, bbox=[1320. 1065. 1419. 1121.]),
   SingleCellStatic(id=8f9f4905-8263-4d8d-94ad-056b50298bb2, timeframe=80, bbox=[1315. 1060. 1427. 1134.]),
   SingleCellStatic(id=f02fa009-a668-4402-b61f-0bc57b82106d, timeframe=79, bbox=[1308. 1054. 1414. 1126.]),
   SingleCellStatic(id=3cff8ff9-2904-46ed-9eb3-9941b0b51e43, timeframe=78, bbox=[1310. 1055. 1411. 1124.]),
   SingleCellStatic(id=02c4ae21-ec3f-45fe-9458-437315eb11dd, timeframe=77, bbox=[1307. 1059. 1398. 1128.]),
   SingleCellStat

Automatically prepare normal samples

require tracking done

In [4]:
# get all scs from class_samples not in normal class
exclude_scs = []
total_non_normal_samples = 0
for class_name, samples in class_samples.items():
    if class_name != "normal":
        for sample in samples:
            exclude_scs.extend(sample)
            total_non_normal_samples += 1

exclude_scs = set(exclude_scs)

load all scs

In [5]:
all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)

In [6]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)
# with open("./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json", "r") as file:
#     json_dict = json.load(file)
# sctc = SingleCellTrajectoryCollection().load_from_json_dict(json_dict)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

In [7]:
objective_sample_num = total_non_normal_samples * 10

normal_frame_len_range = (3, 10)
counter = 0
normal_samples = []
max_trial_counter = 100000
while counter < objective_sample_num and max_trial_counter > 0:
    # randomly select a sct from sctc
    # generate a list of scs
    track_id = np.random.choice(list(sctc.track_id_to_trajectory.keys()))  
    sct = sctc.get_trajectory(track_id)
    # randomly select a length
    frame_len = np.random.randint(*normal_frame_len_range)
    # generate a sample
    times = list(sct.timeframe_to_single_cell.keys())
    times = sorted(times)
    if len(times) <= frame_len:
        continue
    start_idx = np.random.randint(0, len(times) - frame_len)
    start_time = times[start_idx]
    end_time = times[start_idx + frame_len - 1]

    sub_sct = sct.subsct(start_time, end_time)

    is_some_sc_in_exclude_scs = False
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        if sc in exclude_scs:
            is_some_sc_in_exclude_scs = True
            break
    if is_some_sc_in_exclude_scs:
        continue
    
    new_sample = []
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        new_sample.append(sc)
    normal_samples.append(new_sample)
    counter += 1
    max_trial_counter -= 1

normal_samples[:2]

[[SingleCellStatic(id=c44e0f0a-d7af-470b-9783-6ca8233d4aa7, timeframe=220, bbox=[1481  875 1540  967]),
  SingleCellStatic(id=ebd48344-611b-42d1-82cb-e7c902070fe2, timeframe=224, bbox=[1472  871 1552  951]),
  SingleCellStatic(id=828c8e88-3c69-4e5f-b4f9-f0398d78eb07, timeframe=225, bbox=[1476  866 1561  950]),
  SingleCellStatic(id=396a2c33-5bf8-48b0-813c-59efdfda087a, timeframe=226, bbox=[1484  854 1564  963]),
  SingleCellStatic(id=297ecfe4-9755-492e-9438-0cee4c55f0f0, timeframe=227, bbox=[1482  867 1556  960]),
  SingleCellStatic(id=01eb2fa1-0aac-41aa-b79c-b51aad55f5dd, timeframe=228, bbox=[1483  875 1551  951])],
 [SingleCellStatic(id=f1746002-2629-49bb-8231-385b0e93df35, timeframe=245, bbox=[ 571 1856  655 1945]),
  SingleCellStatic(id=008a4add-9b79-4863-93bd-97f7a19c7e74, timeframe=246, bbox=[ 578 1859  672 1948]),
  SingleCellStatic(id=89a61a64-7946-48d6-af6f-51bbfe0c02f2, timeframe=247, bbox=[ 577 1862  660 1950]),
  SingleCellStatic(id=bbac7a7a-4f23-451e-b585-bdd2618f7fb2, tim

In [8]:
class_samples["normal"].extend(normal_samples)

## Prepare videos and annotations for MMDetection

In [9]:
classes = class_samples.keys()
classes

dict_keys(['mitosis', 'apoptosis', 'normal'])

In [10]:
from livecell_tracker.core.utils import gray_img_to_rgb, rgb_img_to_gray
from livecell_tracker.preprocess.utils import normalize_img_to_uint8

In [11]:
[len(sample) for sample in normal_samples]

[6,
 8,
 9,
 4,
 6,
 3,
 4,
 6,
 3,
 9,
 3,
 4,
 9,
 6,
 4,
 9,
 7,
 3,
 7,
 5,
 8,
 8,
 4,
 8,
 3,
 6,
 5,
 7,
 4,
 7,
 6,
 5,
 3,
 9,
 3,
 3,
 8,
 8,
 3,
 3,
 6,
 8,
 6,
 4,
 9,
 6,
 6,
 8,
 9,
 6,
 8,
 8,
 4,
 6,
 6,
 8,
 3,
 5,
 9,
 8,
 3,
 6,
 8,
 5,
 7,
 8,
 6,
 5,
 4,
 9,
 6,
 9,
 6,
 3,
 6,
 3,
 4,
 7,
 3,
 9,
 5,
 3,
 3,
 8,
 7,
 5,
 6,
 8,
 4,
 9,
 5,
 4,
 9,
 8,
 4,
 9,
 7,
 3,
 3,
 7,
 9,
 7,
 4,
 6,
 6,
 8,
 4,
 3,
 7,
 7,
 5,
 6,
 7,
 4,
 7,
 9,
 7,
 4,
 4,
 6]

In [12]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

In [13]:
from typing import List
import cv2
import numpy as np
import pandas as pd

def gen_mp4_from_frames(video_frames, output_file, fps):
    # Define the output video file name and properties
    frame_size = video_frames[0].shape[:2][::-1]  # reverse the order of width and height

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_file), fourcc, fps, frame_size)
    # Write each frame to the output video
    for frame in video_frames:
        out.write(frame)
    out.release()

def gen_samples_mp4s(sc_samples: List[List[SingleCellStatic]], class_label, output_dir, fps = 1, padding_pixels=50):
    """
    Generate mp4 videos and masks from a list of SingleCellStatic samples.
    Args:
        sc_samples: A list of SingleCellStatic samples.
        class_label: A string representing the class label of the samples.
        output_dir: A Path object representing the directory to save the generated videos and masks.
        fps: An integer representing the frames per second of the generated videos.
        padding_pixels: An integer representing the number of pixels to pad around the cells in the generated videos and masks.
    Returns:
        A dictionary containing the file paths of the generated videos, masks, and combined videos.
    """
    res_paths = {
        "video": [],
        "mask": [],
        "combined": []
    }
    for i, sample in enumerate(sc_samples):
        output_file = output_dir / (f'{class_label}_{i}_raw_padding-{padding_pixels}.mp4')
        mask_output_file = output_dir / (f'{class_label}_{i}_mask_padding-{padding_pixels}.mp4')
        combined_output_file = output_dir / (f'{class_label}_{i}_combined_padding-{padding_pixels}.mp4')
        
        # record video file path and class label
        video_frames, video_frame_masks = video_frames_and_masks_from_sample(sample, padding_pixels=padding_pixels)
        combined_frames = combine_video_frames_and_masks(video_frames, video_frame_masks)

        # for debug
        print("len video_frames: ", len(video_frames))
        print("len masks video: ", len(video_frame_masks))
        print("len combined_frames: ", len(combined_frames))

        gen_mp4_from_frames(video_frames, output_file, fps=fps)
        gen_mp4_from_frames(video_frame_masks, mask_output_file, fps=fps)
        gen_mp4_from_frames(combined_frames, combined_output_file, fps=fps)
        res_paths["video"].append(output_file)
        res_paths["mask"].append(mask_output_file)
        res_paths["combined"].append(combined_output_file)
    return res_paths


ver = 4
data_dir = Path(f'notebook_results/mmaction_train_data_v{ver}')
class_labels = ['mitosis', 'apoptosis', 'normal']
class_label = "mitosis"
frame_types = ["video", "mask", "combined"]
selected_frame_type = "video" 

padding_pixels = [0, 20, 40, 50, 100]
mmaction_df = pd.DataFrame(columns=["path", "label_index", "padding_pixels", "frame_type"])
for class_label in class_labels:
    output_dir = Path(data_dir) / "videos"
    output_dir.mkdir(exist_ok=True, parents=True)
    video_frames_samples = class_samples[class_label]
    for padding_pixel in padding_pixels:
        res_paths = gen_samples_mp4s(video_frames_samples, class_label, output_dir, padding_pixels=padding_pixel)
        for selected_frame_type in frame_types:
            mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)

data_df_path = data_dir/'all_data.txt'
mmaction_df.to_csv(data_df_path, index=False, header=False, sep=' ')


for selected_frame_type in frame_types:
    selected_frame_type_df = mmaction_df[mmaction_df["frame_type"] == selected_frame_type]
    train_df_path = data_dir/f'train_data_{selected_frame_type}.txt'
    test_df_path = data_dir/f'test_data_{selected_frame_type}.txt'

    train_df = selected_frame_type_df.sample(frac=0.8, random_state=0, replace=False)
    test_df = selected_frame_type_df.drop(train_df.index, inplace=False)

    # only keep the path and label_index columns
    train_df = train_df[["path", "label_index"]]
    test_df = test_df[["path", "label_index"]]

    train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
    test_df.to_csv(test_df_path, index=False, header=False, sep=' ')


len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  13
len masks video:  13
len combined_frames:  13
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  14
len masks video:  14
len combined_frames:  14
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  10
len masks video:  10
len combined_frames:  10


C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: 

len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  13
len masks video:  13
len combined_frames:  13
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  14
len masks video:  14
len combined_frames:  14
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  10
len masks video:  10
len combined_frames:  10


C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: 

len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  13
len masks video:  13
len combined_frames:  13
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  14
len masks video:  14
len combined_frames:  14
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  12
len masks video:  12
len combined_frames:  12
len video_frames:  11
len masks video:  11
len combined_frames:  11
len video_frames:  10
len masks video:  10
len combined_frames:  10
len video_frames:  10
len masks video:  10
len combined_frames:  10


...

C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: 

len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  8
len masks video:  8
len combined_frames:  8
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  9
len 

C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: 

len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  8
len masks video:  8
len combined_frames:  8
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  3
len masks video:  3
len combined_frames:  3
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  9
len masks video:  9
len combined_frames:  9
len video_frames:  6
len masks video:  6
len combined_frames:  6
len video_frames:  4
len masks video:  4
len combined_frames:  4
len video_frames:  9
len 

C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)
C:\Users\dummy\AppData\Local\Temp\ipykernel_64312\3675440195.py:74: 

In [14]:
# from sklearn.model_selection import train_test_split

# train_df_path = data_dir/'train_data.csv'
# test_df_path = data_dir/'test_data.csv'

# # split train and test from df
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
# test_df.to_csv(test_df_path, index=False, header=False, sep=' ')
